# Simple Fight 1v1 v102
The current implementation is improved version of [simple-fight-1v1-101](./simple-fight-1v1-101.ipynb) with interactive spreadsheet-like interface.

In [ ]:
%pip -q install ipysheet
%pip -q install pandas
%pip install nbimporter

In [6]:

import ipysheet

import pandas as pd
import os
import csv
from collections import namedtuple

import nbimporter
import simple_fight_1v1


In [3]:
DEFAULT_UNIT_TYPES = ['soldier', 'knight', 'archer', 'goblin', 'ork', 'axe-thrower']
ATTRIBUTE_NAMES = ['type', 'health', 'attack_damage', 'speed', 'attack_range', 'attack_cooldown']
UnitAttributes = namedtuple('UnitAttributes', ATTRIBUTE_NAMES)

# we need to check if the local file "simle-fight-1v1-data.csv" exists
# if not, we need to create it and initialize it with some data
DATA_FILE_NAME = 'work_data/simple-fight-1v1-data.csv'

def create_clear_data_file(file_name):
    with open(file_name, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(ATTRIBUTE_NAMES)
        for unit_type in DEFAULT_UNIT_TYPES:
            writer.writerow([unit_type, 100, 10, 10, 10, 1])

if not os.path.exists(DATA_FILE_NAME):
    create_clear_data_file(DATA_FILE_NAME)

# Now we read the data from the file and write down them into pandas DataFrame
df = pd.read_csv(DATA_FILE_NAME, index_col='type')
sheet = ipysheet.from_dataframe(df)
# ipysheet.cell_range(sheet.cells)
def update_data_file(change):
    ipysheet.to_dataframe(sheet).to_csv(DATA_FILE_NAME, index='type', index_label='type')

# Have no idea why but cells are columns in ipysheet and act as cell ranges and we can put observers on them
for cell in sheet.cells:
    cell.observe(update_data_file, 'value')
sheet


Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format='0[.]0', row_end=5, row_start=0, squeeze_row=Fa…

In [27]:
# Now we need to create a function to get unit attributes from the sheet
unit_types = sheet.row_headers

units_data = {}

# Units attributes
# UNITS_ATTRIBUTES = {
#     'soldier': UnitAttributes(health=100, attack_damage=10, speed=1.5, attack_range=1, attack_cooldown=0.7),
#     'knight': UnitAttributes(health=150, attack_damage=15, speed=1.2, attack_range=1, attack_cooldown=1),
#     'archer': UnitAttributes(health=50, attack_damage=20, speed=1, attack_range=10, attack_cooldown=2),
#     'goblin': UnitAttributes(health=50, attack_damage=10, speed=1.5, attack_range=1, attack_cooldown=0.5),
#     'ork': UnitAttributes(health=100, attack_damage=15, speed=1.2, attack_range=1, attack_cooldown=1),
#     'axe-thrower': UnitAttributes(health=50, attack_damage=20, speed=1, attack_range=10, attack_cooldown=2),
# }

for row, unit_type in enumerate(unit_types):
    unit = {}
    for col, attribute in enumerate(ATTRIBUTE_NAMES[1:]):
        unit[attribute] = sheet.cells[col].value[row]
        unit["type"] = unit_type
    units_data[unit_type] = UnitAttributes(**unit)

units_data


{'soldier': Unit(type='soldier', health=33, attack_damage=20, speed=22, attack_range=10, attack_cooldown=1),
 'knight': Unit(type='knight', health=100, attack_damage=10, speed=10, attack_range=10, attack_cooldown=1),
 'archer': Unit(type='archer', health=100, attack_damage=10, speed=10, attack_range=10, attack_cooldown=1),
 'goblin': Unit(type='goblin', health=100, attack_damage=10, speed=10, attack_range=10, attack_cooldown=1),
 'ork': Unit(type='ork', health=100, attack_damage=10, speed=10, attack_range=10, attack_cooldown=1),
 'axe-thrower': Unit(type='axe-thrower', health=100, attack_damage=10, speed=10, attack_range=10, attack_cooldown=1)}

In [ ]:
DT = 0.1
FIELD_SIZE = 15

def create_unit(unit_type: str, x=0) -> dict:
    attributes = units_data[unit_type]
    unit = {}
    unit['type'] = unit_type
    unit['max_health'] = attributes.health
    unit['current_health'] = attributes.health
    unit['current_attack_cooldown'] = 0
    unit['x'] = x
    return unit

def get_unit_attribute(unit: dict, attribute: str):
    return getattr(units_data[unit['type']], attribute)

def distance_between(unit1: dict, unit2: dict):
    return abs(unit1['x'] - unit2['x'])

def enemy_in_range(unit: dict, enemy: dict):
    return distance_between(unit, enemy) <= get_unit_attribute(unit, "attack_range")

def unit_can_attack(unit: dict):
    return unit["current_attack_cooldown"] <= 0

def simulate_fight(unit_type1: dict, unit_type2: dict):
    unit1 = create_unit(unit_type1, 0)
    unit2 = create_unit(unit_type2, FIELD_SIZE)
    time = 0
    while unit1["current_health"] > 0 and unit2["current_health"] > 0:
        if enemy_in_range(unit1, unit2):
            if unit_can_attack(unit1):
                unit2["current_health"] -= get_unit_attribute(unit1, "attack_damage")
                unit1["current_attack_cooldown"] = get_unit_attribute(unit1, "attack_cooldown")
        else:
            unit1["x"] += get_unit_attribute(unit1, "speed") * DT
        if enemy_in_range(unit2, unit1):
            if unit_can_attack(unit2):
                unit1["current_health"] -= get_unit_attribute(unit2, "attack_damage")
                unit2["current_attack_cooldown"] = get_unit_attribute(unit2, "attack_cooldown")
        else:
            unit2["x"] -= get_unit_attribute(unit2, "speed") * DT
        time += DT
        unit1["current_attack_cooldown"] = max(0, unit1["current_attack_cooldown"] - DT)
        unit2["current_attack_cooldown"] = max(0, unit2["current_attack_cooldown"] - DT)
    return time, unit1, unit2